# 系列ラベリング問題とは？

* データの系列$ \bf{x}=\{x_1,x_2,...,x_n\} $を入力として、ラベルの系列$ \bf{y}=\{y_1,y_2,...,y_n\} $を出力とする問題のこと
* $ y_iとy_{i+1} $にはある種の関係があり、$ y_1,y_2,...,y_n $を独立に考えることができない
* 「私は秋田犬が大好き。」
* 「私」「は」「秋田」「犬」「が」「大好き」「。」が$ x_1,x_2,...,x_7 $に対応
* 「名詞」「助詞」「名詞」「名詞」「助詞」「名詞」「記号」が$ y_1,y_2,...,y_7 $に対応
* 単語分割→各文字に対してその文字の後に単語区切りの記号を入れる(ラベル1)か入れない(ラベル0)かのラベルを付与する問題とすることで、系列ラベリング問題と見なせる
* 固有表現抽出→あるカテゴリーを持つ単語や複合語に対してラベルを付与する
  * 人名(岸田文雄を例に挙げる)→人名の開始のラベル(岸田)：B-Personを付与、人名内のラベル(文雄)：I-Personを付与

### ディープラーニング出現前の系列ラベリング問題の解法
#### まず解決すべき課題
* 個々のデータ$ x_i $をどのようなベクトルで表現するか→対象のタスクを解くのに有効そうな特徴を次元に対応させてベクトル化する
* $ \bf{y} $ = $ f(\bf{x}) $となるような関数$ f $をどのように構築するか（学習）
* 関数$ f $を利用して入力系列$ \bf{x} $から$ f(\bf{x}) $をどのように計算するか（推論）

#### HMM(Hidden Markov Model、隠れマルコフモデル)
* 有限次元オートマトンに確率を付与したモデル
* 品詞タガ－(「S(開始記号)」「E(終了記号)」「名詞」「動詞」「副詞」「助詞」)を用いる
* SからEまでに出現する単語$ w_0,w_1,...,w_m $が観測され、これらがどのような状態をたどってきたかを推定し、各単語$ w_i $に品詞を付与する
* 状態$ S_i $から状態$ S_j $に移動する確率$ p(S_j|S_i) $とその移動する際に単語$ w $を出力する確率$ p(w|S_i) $を設定→EMアルゴリズムによって学習を行う
* $ p(w|S_i) $が最大となるような状態の列を推定したい→組み合わせ最適化問題となる

#### CRF(Conditional Random Field、条件付確率場)
* 対数線形モデルの一種
* $ p(\bf{y}|\bf{x})=\frac{1}{Z_{x,w}} e^{w ・\phi(\bf{x}, \bf{y})} $　$ w $：重みベクトル
* $ Z_{x,w}=\sum_{y} e^{w ・ \phi(\bf{x},\bf{y})} $
* $ \phi(\bf{x}, \bf{y}) $：$ \bf{x}, \bf{y} $を入力として、K種類の素性、つまりK次元ベクトルを出力する関数
* $ \phi(\bf{x}, \bf{y})=(\phi_{1}(\bf{x}, \bf{y}), \phi_{2}(\bf{x}, \bf{y}), ..., \phi_{K}(\bf{x}, \bf{y})) $
* $ \phi_k(\bf{x}, \bf{y})=\sum_t{\phi_k(\bf{x}, t, y_t, y_{t-1})} $で$ \phi_k $を限定(Linear-chain CRF)
* $ \sum_t{(w ・ \phi_k(\bf{x}, t, y_t, y_{t-1}))} $の最大値が系列$ \bf{y} $の推定値となる
* $ \sum_t{\phi_k(\bf{x}, t, y_t, y_{t-1})} $：$ x_t $とそのラベル$ y_t $の関係が成立していたら1、成立していなければ0を返す関数　素性関数

|  表記  |  品詞  |  ラベル  |
| ---- | ---- | ---- |
|  田中  |  名詞  |  B-Person  |
|  さん  |  接尾  |  O  |
|  は  |  助詞  |  O  |
|  茨城  |  名詞  |  B-Organization  |
|  大学  |  名詞  |  I-Organization  |
|  の  |  助詞  |  O  |
|  学生  |  名詞  |  O  |
|  です  |  助動詞  |  O  |
|  。  |  句点  |  O  |

* $ \bf{x}_4 $：[表記：茨城, 品詞：名詞], $ y_4 $：B-Organization
* $ \phi_1 $を「表記が"茨城"かつラベルがB-Organization」という条件に設定したとき、条件を満たすのは$ \phi_1(\bf{x}, 4, y_3, y_2) $(4行目)のみとなるため、$ \phi_1(\bf{x}, \bf{y})=1 $となる
* $ \phi_2 $を「品詞が"助詞"かつラベルがO」という条件に設定したとき、条件を満たすのは$ \phi_2(\bf{x}, 3, y_3, y_2) $と$ \phi $(3行目)と$ \phi_2(\bf{x}, 6, y_6, y_5) $(6行目)となるため、$ \phi_2(\bf{x}, \bf{y})=2 $となる
* このような操作を繰り返して$ \phi(\bf{x}, \bf{y}) $を作成する

* HMM

In [1]:
# 品詞タガ－の作成
pos = ['S', 'E', '名詞', '動詞', '副詞', '助詞']  #ラベルとなる品詞
vocab = ['S', 'E', '私', '犬', 'は', 'を', '愛す', '飼う', '去年', 'いつも']  #入力系列の要素となる単語

* $ p(S_j|S_i) $の初期値設定
* $ p(助詞|名詞)=p(S_5|S_2) $は3行6列目の要素→0.6

In [2]:
import numpy as np
t_mat = np.array([
    [0.0, 0.0, 0.6, 0.0, 0.4, 0.0],
    [1.0, 0.0, 0.0, 0.0, 0.0, 0.0],
    [0.0, 0.0, 0.0, 0.0, 0.4, 0.6],
    [0.0, 1.0, 0.0, 0.0, 0.0, 0.0],
    [0.0, 0.0, 0.4, 0.4, 0.2, 0.0],
    [0.0, 0.0, 0.3, 0.4, 0.3, 0.0]        
])

* $ p(w|S_i) $の初期値設定
* $ p(犬|名詞)=p(w_3|S_2) $は3行4列目の要素→0.5

In [3]:
e_mat = np.array([
    [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
    [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
    [0.0, 0.0, 0.5, 0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.5, 0.0, 0.0],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.5],
    [0.0, 0.0, 0.0, 0.0, 0.5, 0.5, 0.0, 0.0, 0.0, 0.0]    
])

In [4]:
pip install hmmlearn

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [5]:
from hmmlearn import hmm
model = hmm.MultinomialHMM(n_components=len(pos),
                           transmat_prior=t_mat,
                           params='e',
                           init_params='t',
                           n_iter=20)
model.emissionprob_ = e_mat

In [6]:
s1 = "S 私 は 犬 を 愛す E"
s2 = "S 去年 犬 を 飼う E"
s3 = "S 犬 は 私 を 愛す E"
s4 = "S 去年 私 は いつも 犬 を 飼う E"
s5 = "S いつも いつも 犬 を 愛す E"

In [7]:
X, lens = [], []
for s in [s1, s2, s3, s4, s5]:
        slist = s.split()
        X += [ vocab.index(w) for w in  slist]
        lens.append(len(slist))
X = np.array(X).reshape(-1,1)

In [8]:
model.fit(X, lengths=lens)

Fitting a model with 54 free scalar parameters with only 36 data points will result in a degenerate solution.


MultinomialHMM(init_params='t', n_components=6, n_iter=20, params='e',
               random_state=RandomState(MT19937) at 0x221EF821340,
               transmat_prior=array([[0. , 0. , 0.6, 0. , 0.4, 0. ],
       [1. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0.4, 0.6],
       [0. , 1. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0.4, 0.4, 0.2, 0. ],
       [0. , 0. , 0.3, 0.4, 0.3, 0. ]]))

In [9]:
ts = "S 犬 は いつも 私 を 愛す E"

In [10]:
ts = [ vocab.index(w) for w in ts.split() ]
ts = np.array(ts).reshape(-1,1)

In [11]:
ans = model.predict(ts)
print([pos[p] for p in ans ])

['S', '名詞', '助詞', '副詞', '名詞', '助詞', '動詞', 'E']


* CRF

In [12]:
pip install sklearn_crfsuite

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [13]:
from nltk.corpus.reader import ConllCorpusReader
train0 = ConllCorpusReader('conll2003', 
                       'train.txt', 
                       ['words', 'pos', 'ignore', 'chunk'])
train_sents = list(train0.iob_sents())
test0 = ConllCorpusReader('conll2003', 
                       'test.txt', 
                       ['words', 'pos', 'ignore', 'chunk'])
test_sents = list(test0.iob_sents())

In [14]:
len(train_sents)

14987

In [15]:
len(test_sents)

3684

In [16]:
# 単語表記、品詞、固有表現のタグの3組からなるタプルのリスト
print(train_sents[1])

[('EU', 'NNP', 'B-ORG'), ('rejects', 'VBZ', 'O'), ('German', 'JJ', 'B-MISC'), ('call', 'NN', 'O'), ('to', 'TO', 'O'), ('boycott', 'VB', 'O'), ('British', 'JJ', 'B-MISC'), ('lamb', 'NN', 'O'), ('.', '.', 'O')]


In [17]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]
    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),  #単語を小文字にする処理
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),  #wordが大文字であるかをブーリアンで返す処理
        'word.istitle()': word.istitle(),  #wordが先頭だけ大文字であるかをブーリアンで返す処理
        'word.isdigit()': word.isdigit(),  #wordが数字であるかをブーリアンで返す処理
        'postag': postag,
        'postag[:2]': postag[:2],
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True
    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True
    return features

In [18]:
def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]
def sent2labels(sent):
    return [label for token, postag, label in sent]
def sent2tokens(sent):
    return [token for token, postag, label in sent]

In [19]:
X_train = [sent2features(s) for s in train_sents]
y_train = [sent2labels(s) for s in train_sents]
X_test = [sent2features(s) for s in test_sents]
y_test = [sent2labels(s) for s in test_sents]

In [20]:
len(X_train)

14987

In [21]:
len(X_train[1])  #EU rejects German call to boycott British lamb.

9

In [22]:
X_train[1][2]  #X_train[1]の2番目の単語"German"に関する情報、CRFの素性

{'bias': 1.0,
 'word.lower()': 'german',
 'word[-3:]': 'man',
 'word[-2:]': 'an',
 'word.isupper()': False,
 'word.istitle()': True,
 'word.isdigit()': False,
 'postag': 'JJ',
 'postag[:2]': 'JJ',
 '-1:word.lower()': 'rejects',
 '-1:word.istitle()': False,
 '-1:word.isupper()': False,
 '-1:postag': 'VBZ',
 '-1:postag[:2]': 'VB',
 '+1:word.lower()': 'call',
 '+1:word.istitle()': False,
 '+1:word.isupper()': False,
 '+1:postag': 'NN',
 '+1:postag[:2]': 'NN'}

In [23]:
import sklearn_crfsuite
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,  #L1正則化パラメータ
    c2=0.1,  #L2正則化パラメータ
    max_iterations=100,
    all_possible_transitions=True
)

In [33]:
# crf.fit(X_train, y_train)

In [34]:
len(crf.state_features_)  #重みベクトルの情報、素性の種類数

25680

In [35]:
crf.state_features_[('word.lower():german', 'B-MISC')]  #素性word.lower():germanとB-MISCの対に対する重み

4.712476

In [32]:
crf.score(X_test, y_test)  #テストデータに対して固有表現抽出を行う

0.9564337245612146

In [36]:
y_pred = crf.predict(X_test)  #ラベル'O'を含めずにモデルを評価するためにpredictを用いる、最終的なラベルの系列を出力

In [37]:
from sklearn_crfsuite import metrics

labels = list(crf.classes_)
labels.remove('O')

f1 = metrics.flat_f1_score(y_test, y_pred, average='weighted', labels=labels)  #F値の算出

In [38]:
f1

0.8014153821544218

In [39]:
ys_pred = crf.predict_marginals(X_test)  #単語の各ラベルに対する確率を算出

In [40]:
ys_pred[1][0]  #テストデータの1番目の文の最初の単語

{'B-ORG': 0.00045332666293954467,
 'O': 0.9984552751743301,
 'B-MISC': 6.724150177606486e-05,
 'B-PER': 0.000219912398209062,
 'I-PER': 2.3138879491281777e-08,
 'B-LOC': 0.0007964142557493604,
 'I-ORG': 4.3541281190583085e-06,
 'I-MISC': 2.40505222223752e-06,
 'I-LOC': 1.0476877758934262e-06}

In [ ]:
# sorted_labels = sorted(labels, key=lambda name: (name[1:], name[0]))
# print(metrics.flat_classification_report(y_test, y_pred, labels=sorted_labels, digits=3))

* LSTM

In [41]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.optim as optim

torch.manual_seed(1)

def argmax(vec):
    # return the argmax as a python int
    _, idx = torch.max(vec, 1)
    return idx.item()


def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)


# Compute log sum exp in a numerically stable way for the forward algorithm
def log_sum_exp(vec):
    max_score = vec[0, argmax(vec)]
    max_score_broadcast = max_score.view(1, -1).expand(1, vec.size()[1])
    return max_score + \
        torch.log(torch.sum(torch.exp(vec - max_score_broadcast)))

class BiLSTM_CRF(nn.Module):

    def __init__(self, vocab_size, tag_to_ix, embedding_dim, hidden_dim):
        super(BiLSTM_CRF, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        self.tag_to_ix = tag_to_ix
        self.tagset_size = len(tag_to_ix)

        self.word_embeds = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim // 2,
                            num_layers=1, bidirectional=True)

        # Maps the output of the LSTM into tag space.
        self.hidden2tag = nn.Linear(hidden_dim, self.tagset_size)

        # Matrix of transition parameters.  Entry i,j is the score of
        # transitioning *to* i *from* j.
        self.transitions = nn.Parameter(
            torch.randn(self.tagset_size, self.tagset_size))

        # These two statements enforce the constraint that we never transfer
        # to the start tag and we never transfer from the stop tag
        self.transitions.data[tag_to_ix[START_TAG], :] = -10000
        self.transitions.data[:, tag_to_ix[STOP_TAG]] = -10000

        self.hidden = self.init_hidden()

    def init_hidden(self):
        return (torch.randn(2, 1, self.hidden_dim // 2),
                torch.randn(2, 1, self.hidden_dim // 2))

    def _forward_alg(self, feats):
        # Do the forward algorithm to compute the partition function
        init_alphas = torch.full((1, self.tagset_size), -10000.)
        # START_TAG has all of the score.
        init_alphas[0][self.tag_to_ix[START_TAG]] = 0.

        # Wrap in a variable so that we will get automatic backprop
        forward_var = init_alphas

        # Iterate through the sentence
        for feat in feats:
            alphas_t = []  # The forward tensors at this timestep
            for next_tag in range(self.tagset_size):
                # broadcast the emission score: it is the same regardless of
                # the previous tag
                emit_score = feat[next_tag].view(
                    1, -1).expand(1, self.tagset_size)
                # the ith entry of trans_score is the score of transitioning to
                # next_tag from i
                trans_score = self.transitions[next_tag].view(1, -1)
                # The ith entry of next_tag_var is the value for the
                # edge (i -> next_tag) before we do log-sum-exp
                next_tag_var = forward_var + trans_score + emit_score
                # The forward variable for this tag is log-sum-exp of all the
                # scores.
                alphas_t.append(log_sum_exp(next_tag_var).view(1))
            forward_var = torch.cat(alphas_t).view(1, -1)
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        alpha = log_sum_exp(terminal_var)
        return alpha

    def _get_lstm_features(self, sentence):
        self.hidden = self.init_hidden()
        embeds = self.word_embeds(sentence).view(len(sentence), 1, -1)
        lstm_out, self.hidden = self.lstm(embeds, self.hidden)
        lstm_out = lstm_out.view(len(sentence), self.hidden_dim)
        lstm_feats = self.hidden2tag(lstm_out)
        return lstm_feats

    def _score_sentence(self, feats, tags):
        # Gives the score of a provided tag sequence
        score = torch.zeros(1)
        tags = torch.cat([torch.tensor([self.tag_to_ix[START_TAG]], dtype=torch.long), tags])
        for i, feat in enumerate(feats):
            score = score + \
                self.transitions[tags[i + 1], tags[i]] + feat[tags[i + 1]]
        score = score + self.transitions[self.tag_to_ix[STOP_TAG], tags[-1]]
        return score

    def _viterbi_decode(self, feats):
        backpointers = []

        # Initialize the viterbi variables in log space
        init_vvars = torch.full((1, self.tagset_size), -10000.)
        init_vvars[0][self.tag_to_ix[START_TAG]] = 0

        # forward_var at step i holds the viterbi variables for step i-1
        forward_var = init_vvars
        for feat in feats:
            bptrs_t = []  # holds the backpointers for this step
            viterbivars_t = []  # holds the viterbi variables for this step

            for next_tag in range(self.tagset_size):
                # next_tag_var[i] holds the viterbi variable for tag i at the
                # previous step, plus the score of transitioning
                # from tag i to next_tag.
                # We don't include the emission scores here because the max
                # does not depend on them (we add them in below)
                next_tag_var = forward_var + self.transitions[next_tag]
                best_tag_id = argmax(next_tag_var)
                bptrs_t.append(best_tag_id)
                viterbivars_t.append(next_tag_var[0][best_tag_id].view(1))
            # Now add in the emission scores, and assign forward_var to the set
            # of viterbi variables we just computed
            forward_var = (torch.cat(viterbivars_t) + feat).view(1, -1)
            backpointers.append(bptrs_t)

        # Transition to STOP_TAG
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        best_tag_id = argmax(terminal_var)
        path_score = terminal_var[0][best_tag_id]

        # Follow the back pointers to decode the best path.
        best_path = [best_tag_id]
        for bptrs_t in reversed(backpointers):
            best_tag_id = bptrs_t[best_tag_id]
            best_path.append(best_tag_id)
        # Pop off the start tag (we dont want to return that to the caller)
        start = best_path.pop()
        assert start == self.tag_to_ix[START_TAG]  # Sanity check
        best_path.reverse()
        return path_score, best_path

    def neg_log_likelihood(self, sentence, tags):
        feats = self._get_lstm_features(sentence)
        forward_score = self._forward_alg(feats)
        gold_score = self._score_sentence(feats, tags)
        return forward_score - gold_score

    def forward(self, sentence):  # dont confuse this with _forward_alg above.
        # Get the emission scores from the BiLSTM
        lstm_feats = self._get_lstm_features(sentence)

        # Find the best path, given the features.
        score, tag_seq = self._viterbi_decode(lstm_feats)
        return score, tag_seq


START_TAG = "<START>"
STOP_TAG = "<STOP>"
EMBEDDING_DIM = 5
HIDDEN_DIM = 4

# Make up some training data
training_data = [(
    "the wall street journal reported today that apple corporation made money".split(),
    "B I I I O O O B I O O".split()
), (
    "georgia tech is a university in georgia".split(),
    "B I O O O O B".split()
)]

word_to_ix = {}
for sentence, tags in training_data:
    for word in sentence:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)

tag_to_ix = {"B": 0, "I": 1, "O": 2, START_TAG: 3, STOP_TAG: 4}

model = BiLSTM_CRF(len(word_to_ix), tag_to_ix, EMBEDDING_DIM, HIDDEN_DIM)
optimizer = optim.SGD(model.parameters(), lr=0.01, weight_decay=1e-4)

# Check predictions before training
with torch.no_grad():
    precheck_sent = prepare_sequence(training_data[0][0], word_to_ix)
    precheck_tags = torch.tensor([tag_to_ix[t] for t in training_data[0][1]], dtype=torch.long)
    print(model(precheck_sent))

# Make sure prepare_sequence from earlier in the LSTM section is loaded
for epoch in range(
        300):  # again, normally you would NOT do 300 epochs, it is toy data
    for sentence, tags in training_data:
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is,
        # turn them into Tensors of word indices.
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = torch.tensor([tag_to_ix[t] for t in tags], dtype=torch.long)

        # Step 3. Run our forward pass.
        loss = model.neg_log_likelihood(sentence_in, targets)

        # Step 4. Compute the loss, gradients, and update the parameters by
        # calling optimizer.step()
        loss.backward()
        optimizer.step()

# Check predictions after training
with torch.no_grad():
    precheck_sent = prepare_sequence(training_data[0][0], word_to_ix)
    print(model(precheck_sent))

(tensor(2.6907), [1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1])
(tensor(20.4906), [0, 1, 1, 1, 2, 2, 2, 0, 1, 2, 2])


* (略)